get a list of folder topics from bookmarks?


In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var getBookmarksFromTakeout = importer.import('parse bookmarks file');

function getBookmarkFolders() {
    return getBookmarksFromTakeout()
        .then(links => {
            // group by half hour
            const groups = Object.keys(links).sort((a, b) => parseInt(b) - parseInt(a));
            return groups.reduce((acc, t) => {
                const folders = _.groupBy(links[t], l => l.folder);
                acc = acc.concat(Object.keys(folders));
                return acc;
            }, []);
        })
        .then(folders => folders.filter((f, i, arr) => arr.indexOf(f) === i))
}

module.exports = getBookmarkFolders;

if(typeof $$ != 'undefined') {
    $$.async();
    
    getBookmarkFolders()
        .then(r => $$.mime({'text/plain': r.join('\n')}))
        .catch(e => $$.sendError(e))
}


update insterests resume sheet?


In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var getBookmarksFromTakeout = importer.import('parse bookmarks file');
var updateRow = importer.import('update a row in google sheets')
var getDataSheet = importer.import('get google data sheet')

var doc = 'https://docs.google.com/spreadsheets/d/1dAtBQyn5vwStM6ITn7iCpp996Sus26k0bJtbGNlCX2g/edit#gid=257854275';

function updateInterestPage() {
    var links, interests;
    return getDataSheet(doc, 'Interests data')
        .then(rows => interests = rows.map(r => r.category).filter((r, i, arr) => arr.indexOf(r) === i))
        .then(() => getBookmarksFromTakeout())
        .then(l => links = [].concat(...Object.values(l))
              .filter(b => interests.includes(b.folder))
              .map(b => ({
                category: b.folder,
                date: b.time,
                title: b.title,
                link: b.url,
                skills: null,
                tools: null
            })))
        .then(() => importer.runAllPromises(links.map(l => resolve => {
            // category date title link skills tools
            return updateRow(doc, r => {
                return r.category == l.category && !r.link
                             || r.date == l.date && r.link == l.link}, l, 'Interests data')
                .then(resolve);
        })))
}

module.exports = updateInterestPage;

if(typeof $$ != 'undefined') {
    $$.async();
    
    updateInterestPage()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


load message history?

for all inquiries about specific messages in the message history.



In [ ]:
const path = require('path')
const fs = require('fs')
const llama = importer.import('create llm session')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH} = importer.import('express handle conversation')



async function messageHistory(session, prompt) {

  if(!session) {
    return ''
  }

  const {llmPrompt} = await llama
  // TODO: extract time frame to, from prompt
  let history = fs.readdirSync(PROJECT_PATH)
  let historyFiles = []
  for(let i = 0; i < history.length; i++) {
    if(history[i].match('-' + session + '.json')) {
      historyFiles.push(path.basename(history[i]).replace('-' + session + '.json', ''))
    }
  }

  let q1 = 'Current date: ' + (new Date).toISOString()
    + '\nGiven the following chat history:\n'
    + historyFiles
    + '\nGiven the following prompt:\n'
    + prompt + '\nReturn the Year-Month for all related history.'
  console.log('User: ' + q1)
  let a1 = await llmPrompt(q1)
  console.log('AI: ' + a1)

  // TODO: create a range of month out of the response
  for(let i = 0; i < historyFiles.length; i++) {
    if(a1.match(historyFiles[i])) {
      let convoFile = path.join(PROJECT_PATH, historyFiles[i] + '-' + session + '.json')
      if(fs.existsSync(convoFile)) {
        ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
      } else {
        ACTIVE_CONVERSATIONS[convoFile] = {}
      }
    }
  }

  // TODO: search all loaded user histories for keywords
  let keywords = []
  let loadedConversations = Object.keys(ACTIVE_CONVERSATIONS)
    .filter(key => key.match('-' + session + '.json'))
  loadedConversations.sort((a, b) => b - a)
  for(let i = 0; i < loadedConversations.length; i++) {
    let conversation = ACTIVE_CONVERSATIONS[loadedConversations[i]]
    let timestamps = Object.keys(conversation)
    timestamps.sort((a, b) => b - a)
    for(let j = 0; j < timestamps.length; j++) {
      let message = conversation[timestamps[j]]
      let newKeywords = message.keywords
      newKeywords = newKeywords.replace(/.*?:\s*\n\s*/gi, '')
      newKeywords = newKeywords.replaceAll('*', '').replaceAll('# ', '').replaceAll('#', '').replaceAll(/[0-9]+\.\s*/gi, '')
      newKeywords = newKeywords.split(/\s*\n\s*|\s*,\s*/gi)
      keywords = keywords.concat(newKeywords.map(k => k.trim()))
    }
  }
  keywords = keywords.filter((k, i, arr) => k && arr.indexOf(k) == i)


  let q2 = 'Given the following keywords:\n'
    + keywords.join('\n')
    + '\nGiven the following prompt:\n'
    + prompt + '\nReturn all the related keywords.'
  console.log('User: ' + q2)
  let a2 = await llmPrompt(q2)
  console.log('AI: ' + a2)

  let matching = []
  for(let i = 0; i < keywords.length; i++) {
    if(a2.match(keywords[i])) {
      matching.push(keywords[i])
    }
  }

  let count = 0
  let messages = 'Current date: ' + (new Date).toISOString() 
    + '\nOur recent topics:\n'
  for(let i = 0; i < loadedConversations.length && count < 25; i++) {
    let conversation = ACTIVE_CONVERSATIONS[loadedConversations[i]]
    let timestamps = Object.keys(conversation)
    timestamps.sort((a, b) => b - a)
    for(let j = 0; j < timestamps.length && count < 25; j++) {
      let message = conversation[timestamps[j]]
      let topics = matching.filter(key => message.keywords.match(key))
      if(!prompt.match(timestamps[j]) && topics.length == 0) {
        continue
      }

      messages += new Date(parseInt(timestamps[j])).toISOString() + ' - ' + topics.join(', ') + '\n'
      if(messages.length > 2048) {
        break
      }
    }
  }

  

  let q3 = 'Given the following keywords:\n'
    + matching.join('\n')
    + '\n' + messages
    + prompt + '\nAre there any specific related message times?'
  console.log('User: ' + q3)
  let a3 = await llmPrompt(q3)
  console.log('AI: ' + a3)

  messages = 'Current date: ' + (new Date).toISOString() 
  messages += '\nOur most recent conversation:\n'
  
  for(let i = 0; i < loadedConversations.length && count < 25; i++) {
    let conversation = ACTIVE_CONVERSATIONS[loadedConversations[i]]
    let timestamps = Object.keys(conversation)
    timestamps.sort((a, b) => b - a)
    for(let j = 0; j < timestamps.length && count < 25; j++) {
      let message = conversation[timestamps[j]]

      if(!prompt.match(timestamps[j]) && !a3.match(timestamps[j]) && !a3.match(new Date(parseInt(timestamps[j])).toISOString())) {
        continue
      }

      let useSummary = i > 10 || messages.length > 2048 || (message.content && message.content.length > 2048)
      let useKeywords = !message.content
      messages += (message.user ? 'User: ' : 'AI: ') 
        + (useKeywords ? message.keywords : (useSummary ? message.summary : message.content))
        + '\n\n'
      if(messages.length > 4096) {
        break
      }
      count++
    }
    if(messages.length > 4096) {
      break
    }
  }

  for(let i = 0; i < loadedConversations.length && count < 25; i++) {
    let conversation = ACTIVE_CONVERSATIONS[loadedConversations[i]]
    let timestamps = Object.keys(conversation)
    timestamps.sort((a, b) => b - a)
    for(let j = 0; j < timestamps.length && count < 25; j++) {
      let message = conversation[timestamps[j]]

      if(!prompt.match(timestamps[j]) && matching.filter(key => message.keywords.match(key)).length == 0) {
        continue
      }

      let useSummary = i > 10 || messages.length > 2048 || (message.content && message.content.length > 2048)
      let useKeywords = !message.content
      messages += (message.user ? 'User: ' : 'AI: ') 
        + (useKeywords ? message.keywords : (useSummary ? message.summary : message.content))
        + '\n\n'
      if(messages.length > 4096) {
        break
      }
      count++
    }
    if(messages.length > 4096) {
      break
    }
  }

  return messages
}

module.exports = messageHistory



general chit chat?

everything else falls into this category. context will be populated with recent message conversation history.


In [ ]:
const path = require('path')
const fs = require('fs')


const PROJECT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'conversations')
const ACTIVE_CONVERSATIONS = {}

// TODO: load some previous summaries and contents

async function messageRecents(session) {

  if(!session) {
    return ''
  }

  let now = new Date()
  let convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') + '-' + session + '.json')
  // TODO: reload chat
  if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
    if(fs.existsSync(convoFile)) {
      ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
    } else {
      ACTIVE_CONVERSATIONS[convoFile] = {}
    }
  }

  let messageTimes = Object.keys(ACTIVE_CONVERSATIONS[convoFile])
  messageTimes.sort((a, b) => b - a)
  let count = 0
  let messages = 'Current date: ' + (new Date).toISOString() 
    + '\nOur recent conversion:\n'
  for(let i = 0; i < messageTimes.length && count < 25; i++) {
    let message = ACTIVE_CONVERSATIONS[convoFile][messageTimes[i]]
    let useSummary = i > 10 || messages.length > 1024 || (message.content && message.content.length > 1024)
    let useKeywords = !message.content
    messages += (message.user ? 'User: ' : 'AI: ') 
      + (useKeywords ? message.keywords : (useSummary ? message.summary : message.content))
      + '\n\n'
    if(messages.length > 2048) {
      break
    }
    count++
  }

  return messages
}

module.exports = {
  messageRecents,
  PROJECT_PATH,
  ACTIVE_CONVERSATIONS,
  
}



brians resume?

for inquiries about brian cullinan's resume specifically.


In [ ]:
const llama = importer.import('create llm session')
const {selectDom} = importer.import('select tree')

// TODO: populate the context with details from my resume

async function messageResume(session, prompt) {
  const {llmPrompt} = await llama
  let text = selectDom(['//*[text()]', './text()'], importer.interpret('brians resume in html').code)
  let originalPrompt = 'This is part of Brian Cullinan\'s resume:\n'
  let originalLength = originalPrompt.length
  let answers = []
  for(let i = 0; i < text.length; i++) {
    originalPrompt += text[i]
    if(originalPrompt.length > 2048) {
      let q1 = originalPrompt + '\nWrite a short summary of this resume section.'
      console.log('User: ' + q1)
      answers.push(await llmPrompt(q1))
      let a2 = await llmPrompt(originalPrompt 
        + '\nAnswer this prompt, or respond No if it is not related to the question:\n' 
        + prompt + '\nRespond with "No" if the prompt is unrelated.')
      if(a2.trim() != 'No' && a2.trim() != 'No.') {
        answers.push(a2)
      }
      originalPrompt = 'This is part of Brian Cullinan\'s resume:\n'
    }
  }

  if(originalPrompt.length > originalLength) {
    let q1 = originalPrompt + '\nWrite a short summary of this resume section.'
    console.log('User: ' + q1)
    answers.push(await llmPrompt(q1))
    let a2 = await llmPrompt(originalPrompt 
      + '\nAnswer this prompt, or respond No if it is not related to the question:\n' 
      + prompt + '\nRespond with "No" if the prompt is unrelated.')
    if(a2.trim() != 'No' && a2.trim() != 'No.') {
      answers.push(a2)
    }
}

  return answers.join('\n')
}

module.exports = messageResume



file system access?

All queries related to accessing local projects on the file system.


In [ ]:
const path = require('path')
const fs = require('fs')
const llama = importer.import('create llm session')

const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';

async function messageFilesystem(session, prompt) {
  const {llmPrompt} = await llama

  let currentProject = path.basename(path.resolve(__dirname, '..'))
  let availableProjects = fs.readdirSync(PROFILE_PATH)
    .filter(dir => dir[0] != '.')
    .filter(dir => fs.statSync(path.join(PROFILE_PATH, dir)).isDirectory())
    .filter(dir => fs.existsSync(path.join(PROFILE_PATH, dir, '.git')))
  let q1 = 'Current project: ' + currentProject
    + '\nAvailable projects:\n' + availableProjects.join('\n')
    + '\nAre any of these project related to the current prompt:\n'
    + prompt
    + '\nOnly respond with the project name or "No" if none are related.'
  console.log('User: ' + q1)
  let a1 = await llmPrompt(q1)
  console.log('AI: ' + a1)
  
  let relatedProjects = availableProjects.filter(dir => a1.match(dir))

  if(relatedProjects.length == 0) {
    return 'Current project: ' + currentProject
      + '\nAvailable projects:\n' + availableProjects.join('\n')
  }

  let projectFiles = fs.readdirSync(path.join(PROFILE_PATH, relatedProjects[0]))
    .filter(dir => dir[0] != '.' && dir != 'node_modules')
    .map(file => {
      let currentPath = path.join(PROFILE_PATH, relatedProjects[0], file)
      if(fs.statSync(currentPath).isDirectory()) {
        return [file].concat(fs.readdirSync(currentPath)
          .filter(dir => dir[0] != '.')
          .map(dir => path.join(file, dir)))
      } else {
        return file
      }
    })
    .flat(1)
  let relatedFiles = []
  if(currentProject == relatedProjects[0]) {
    // TODO: use cell lookup instead of notebook files
    let q2 = 'Current project: ' + currentProject
      + '\nRelated projects:\n' + relatedProjects.join('\n')
      + '\nProject files:\n'
    let originalFilePrompt = q2
    let count = 0
    for(let i = 0; i < projectFiles.length; i++) {
      q2 += projectFiles[i] + '\n'
      count++
      if(count == 25 || q2.length > 2048) {
        q2 += '\nAre any of the project files relevant to the prompt:\n'
          + prompt
          + '\nOnly respond with the related file paths or file names, nothing else.'
        console.log('User: ' + q2)
        let a2 = await llmPrompt(q2)
        console.log('AI: ' + a2)
        relatedFiles = relatedFiles.concat(projectFiles
          .filter(file => a2.match(file) || a2.match(path.basename(file))))
        count = 0
        q2 = originalFilePrompt
      }
    }
    if(q2.length != originalFilePrompt.length || count > 0) {
      q2 += '\nAre any of the project files relevant to the prompt:\n'
        + prompt
        + '\nOnly respond with the related file paths or file names, nothing else.'
      console.log('User: ' + q2)
      let a2 = await llmPrompt(q2)
      console.log('AI: ' + a2)
      relatedFiles = relatedFiles.concat(projectFiles
        .filter(file => a2.match(file) || a2.match(path.basename(file))))
    }
  }

  if(relatedFiles.length == 0) {
    return 'Current project: ' + currentProject
      + '\nRelated projects:\n' + relatedProjects.join('\n')
      + '\nProject files:\n' + projectFiles.join('\n')
  }

  // TODO: another depth of related files
  
  return 'Current project: ' + currentProject
    + '\nRelated projects:\n' + relatedProjects.join('\n')
    + '\nRelated files:\n' + relatedFiles.join('\n')


}

module.exports = messageFilesystem



add conversation context?


In [ ]:
const getExports = importer.import('get exports from source')
const llama = importer.import('create llm session')
const {messageRecents} = importer.import('general chit chat')


const API_FUNCTIONS = [
  'load message history',
  'general chit chat',
  'brians resume',
  'file system access',
]

async function classifyPrompt(session, prompt) {
  const {llmPrompt} = await llama
  
  let functions = []
  let functionStrings = ''
  for(let i = 0; i < API_FUNCTIONS.length; i++) {
    let code = importer.interpret(API_FUNCTIONS[i])
    functions[i] = getExports(code.code)[0]
    functionStrings += functions[i] + ' - ' + code.markdown.join('').trim().replaceAll(/\s*[\n]+[\s\r\n]*/gi, '-') + '\n'
  }

  console.log(functions)

  let q1 = 'Given the following functions:\n'
    + functionStrings + '\nDo any of these functions relate to this prompt:\n'
    + prompt + '\nOnly return the function name and nothing else.'
  console.log('User: ' + q1)
  let a1 = await llmPrompt(q1)
  console.log('AI: ' + a1)

  let importedFunction
  let answer = ''
  for(let i = 0; i < API_FUNCTIONS.length; i++) {
    if(a1.length > 0 && functions[i].match(a1) || a1.match(functions[i])) {
      importedFunction = importer.import(API_FUNCTIONS[i])
      if(typeof importedFunction == 'object' && typeof Object.values(importedFunction)[0] == 'function') {
        importedFunction = Object.values(importedFunction)[0]
      }
      answer = await importedFunction(session, prompt)
      break
    }
  }

  debugger
  if(importedFunction != messageRecents) {
    return (answer ? answer : '') + await messageRecents(session, prompt)
  } else if (!answer) {
    return await messageRecents(session, prompt)
  } else {
    return answer
  }

}

module.exports = classifyPrompt




express handle conversation?



In [ ]:
const path = require('path')
const fs = require('fs')
const llama = importer.import('create llm session')
const {Remarkable} = require('remarkable');
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
const classifyPrompt = importer.import('add conversation context')
const {askLlamaAboutEmotions} = importer.import('ask llm about emotions')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH} = importer.import('general chit chat')

async function handleConversation(req, res) {
  const {llmPrompt} = await llama
  let now = new Date()
  let convoFile
  let context
  if(req.body.session) {
    convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
      + String(now.getMonth() + 1).padStart(2, '0') + '-' + req.body.session + '.json')
    if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
      if(fs.existsSync(convoFile)) {
        ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
      } else {
        ACTIVE_CONVERSATIONS[convoFile] = {}
      }
    }
  }
  let summary
  if(!req.body.otr) {
    summary = await llmPrompt('Summerize this prompt in one short sentence:\n' 
      + req.body.content + '\nOnly respond with the summary, no pleasantries.')
  }
  let keywords = await llmPrompt('List a few key words that categorize this prompt:\n' 
    + req.body.content + '\nOnly respond with a single category, no pleasantries.')
  let emotions = await askLlamaAboutEmotions(req.body.content)
  context = await classifyPrompt(req.body.session, req.body.content)
  if(req.body.session) {
    ACTIVE_CONVERSATIONS[convoFile][Date.now()] = {
      user: true,
      content: req.body.otr ? void 0 : req.body.content,
      context: context,
      summary: summary,
      keywords: keywords,
      emotions: emotions,
      otr: req.body.otr ? true : false,
    }
    fs.writeFileSync(convoFile, JSON.stringify(ACTIVE_CONVERSATIONS[convoFile], null, 4))
  }

  let result = await llmPrompt((context ? context : '') 
    + '\nRespond to this message and pretend to be emotional (' 
    + emotions.join(', ') + '):\n' 
    + req.body.content)
  let resultSummary
  if(!req.body.otr) {
    resultSummary = await llmPrompt('Summerize this response in one short sentence:\n' 
      + result + '\nOnly respond with the summary, no pleasantries.')
  }
  let resultKeywords = await llmPrompt('List a few key words that categorize this response:\n' 
    + result + '\nOnly respond with a single category, no pleasantries.')
  let resultEmotions = await askLlamaAboutEmotions(result)
  if(req.body.session) {
    ACTIVE_CONVERSATIONS[convoFile][Date.now()+1] = {
      user: false,
      content: req.body.otr ? void 0 : result,
      summary: resultSummary,
      keywords: resultKeywords,
      emotions: resultEmotions,
      otr: req.body.otr ? true : false,
    }
    // TODO: save
    fs.writeFileSync(convoFile, JSON.stringify(ACTIVE_CONVERSATIONS[convoFile], null, 4))
  }
  const mdHtml = md.render(result);
  res.send(mdHtml)
}

module.exports = handleConversation




render message history?


In [ ]:
const path = require('path')
const fs = require('fs')
const {Remarkable} = require('remarkable');
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
const {ACTIVE_CONVERSATIONS, PROJECT_PATH} = importer.import('general chit chat')

function renderMessages(req, res) {
  let now = new Date()
  let when = now
  if(req.query.when) {
    when = new Date(parseInt(req.query.when))
  }
  let from = new Date(0)
  if(req.query.from) {
    from = new Date(parseInt(req.query.from))
  }
  //console.log(when.getTime(), from.getTime())
  if(!req.query.session) {
    return res.send(importer.interpret('resume chat program').code)
  }
  let convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') + '-' + req.query.session + '.json')
  // TODO: reload chat
  if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
    if(fs.existsSync(convoFile)) {
      ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
    } else {
      ACTIVE_CONVERSATIONS[convoFile] = {}
    }
  }
  let messageTimes = Object.keys(ACTIVE_CONVERSATIONS[convoFile])
  messageTimes.sort((a, b) => b - a)
  let compareWhen = when.getTime()
  let compareFrom = from.getTime()
  let count = 0
  let messages = ''
  for(let i = 0; i < messageTimes.length && count < 25; i++) {
    if(parseInt(messageTimes[i]) > compareWhen || parseInt(messageTimes[i]) < compareFrom) {
      continue
    }

    let message = ACTIVE_CONVERSATIONS[convoFile][messageTimes[i]]
    // load messages in reverse order because the are sorted from recent to oldest
    if(typeof message.content != 'undefined') {
      if(message.user) {
        messages = '<div id="id-' + messageTimes[i] + '" class="user"><p>User: ' + message.content + '</p></div>\n' + messages
      } else {
        const mdHtml = md.render(message.content);
        messages = '<div id="id-' + messageTimes[i] + '" class="ai">AI: ' + mdHtml + '</div>\n' + messages
      }
    } else {
      if(message.user) {
        messages = '<div id="id-' + messageTimes[i] + '" class="user"><p>User: Content off the record</p></div>\n' + messages
      } else {
        messages = '<div id="id-' + messageTimes[i] + '" class="ai">AI: Content off the record</div>\n' + messages
      }
    }
    count++
  }
  return res.send(importer.interpret('resume chat program').code
    .replace('<div id="messages"></div>', '<div id="messages">' + messages + '</div>'))
}

module.exports = {
  renderMessages
}



resume express chat service?


In [ ]:
const {doClick, doKeys, getMonitor, serveHomepage} = importer.import('express automation routes')
const handleConversation = importer.import('express handle conversation')
const {renderMessages} = importer.import('render message history')

const fs = require('fs')
const express = require('express')
const cors = require('cors')
const mime = require('mime')
const path = require('path')

async function startServer() {
  var app = express()
  app.use(cors({
    credentials: true,
    origin: '*' 
  }))
  app.use(express.json())
  app.get('/monitors/*', getMonitor)
  app.post('/', handleConversation)
  app.get('/embed', renderMessages)
  app.get('/', function (req, res) {
    res.send(importer.interpret('chat page').code)
  })

  let port = process.env.CHAT_PORT || 8181
  app.listen(port, () => console.log('Listening on https://localhost:' + port))
}

module.exports = startServer



resume chat program?


In [ ]:

<div id="chat-inner">
  <label for="otr"><input type="checkbox" id="otr" value="otr" />Off the record</label>
  <div id="text-entry">
    <textarea placeholder="Enter your message"></textarea>
    <button>Send</button>
  </div>
  <div id="messages"></div>
</div>




chat page?


In [ ]:
<html>

<head>
  <style>
    html,
    body {
      padding: 0;
      margin: 0;
      background: rgb(51, 51, 68);
      color: white;
      font-family: Verdana, Geneva, Tahoma, sans-serif;
    }

    div.container {
      width: auto;
      padding: 0;
      background: #222;
      box-shadow: rgb(0 0 0 / 50%) 2px 3px 10px;
      margin: 20px 20px 20px 240px;
      border-radius: 2px;
      border: 1px solid #666;
      position: relative;
      z-index: 1;
    }

    #chat-program {
      position: absolute;
      right: 0px;
      left: 0px;
      top: 0px;
      bottom: 0px;
    }

    #chat-inner {
      display: flex;
      flex-direction: column-reverse;
      min-height: 100%;
      height: 100%;
      max-height: 100%;
    }

    #chat-inner #messages {
      height: 100%;
      overflow: auto;
      padding: 20px;
    }

    #chat-inner #messages > div {
      margin: 20px 0px;
    }

    #chat-inner #messages > div.user {
      text-align: right;
      margin: 30px 0px;
    }

    #chat-inner #messages > div.user p {
      padding: 10px;
      background-color: #444;
      display: inline;
      border-radius: 10px;
    }

    #chat-inner #text-entry {
      display: flex;
      border: 2px solid #CCC;
      border-radius: 10px;
      padding: 2px;
    }

    #chat-inner #text-entry textarea {
      width: 100%;
      border: none;
      background-color: #222;
      color: #EEE;
      padding: 3px;
      min-height: 3em;
      max-height: 30em;
      font-size: 18px;
      outline: none;
    }

    #chat-inner #text-entry button {
      background-color: #222;
      border: none;
      color: white;
      font-size: 18px;
      font-weight: bold;
    }
  </style>
</head>

<body>

  <div id="messages-hidden" style="display:none;"></div>

  <div class="container" id="chat-program">


  </div>

  <script>
    let historyIndex = 0
    let currentValue = ''
    function sendHandler(evt) {
      let chatBox = document.querySelector('#chat-program textarea')
      if (event.key === 'Enter') {
        evt.preventDefault()
        let session = localStorage.getItem('brian-chat-session')
        let messageHistory = document.querySelector('#messages')
        let otr = document.querySelector('#chat-program input')
        messageHistory.innerHTML += '<div class="user"><p>User: ' + chatBox.value + '</p></div>'
        let messagePost = {
          method: 'POST',
          body: JSON.stringify({ otr: otr.checked, session: session, content: chatBox.value }),
          headers: {
            "Content-Type": "application/json",
          }
        }
        if (chatBox.value.trim() == '') {
          chatBox.value = ''
          return
        }
        chatBox.value = ''
        historyIndex = 0
        setTimeout(async () => {
          messageHistory.scrollTop = messageHistory.scrollTop + 10000
          let result = await fetch('https://brian-chat.pryor.games/?session=' + session + '&t=' + Date.now(), messagePost)
          if (result.ok) {
            messageHistory.innerHTML += '<div class="ai">AI: ' + await result.text() + '</div>'
          }
          messageHistory.scrollTop = messageHistory.scrollTop + 10000
        }, 100)
        return false
      }
      if (event.key === 'ArrowUp' && chatBox.scrollTop < 10) {
        if (historyIndex === 0) {
          currentValue = chatBox.value
        }
        let messageHistory = document.querySelector('#messages')
        // TODO: adjust according to messages loaded, don't count too high
        let count = 0
        let i
        for (i = messageHistory.childNodes.length - 1; i >= 0 && count < (historyIndex + 1); i--) {
          if (!messageHistory.childNodes[i].innerText) {
            continue
          }
          if (messageHistory.childNodes[i].innerText.startsWith('User:')) {
            count++
          }
        }
        if (i > 0) {
          historyIndex++
          chatBox.value = messageHistory.childNodes[i + 1].innerText.substring(6)
        }
      }
      if (event.key === 'ArrowDown' && historyIndex > 0) {
        let messageHistory = document.querySelector('#messages')
        if (historyIndex === 1) {
          historyIndex--
          chatBox.value = currentValue
        } else {
          let count = 0
          let i
          for (i = messageHistory.childNodes.length - 1; i >= 0 && count < (historyIndex - 1); i--) {
            if (!messageHistory.childNodes[i].innerText) {
              continue
            }
            if (messageHistory.childNodes[i].innerText.startsWith('User:')) {
              count++
            }
          }
          if (i > 0) {
            historyIndex--
            chatBox.value = messageHistory.childNodes[i + 1].innerText.substring(6)
          }
        }
      }
    }

    document.addEventListener('DOMContentLoaded', async (event) => {
      let session = localStorage.getItem('brian-chat-session')

      if (!session) {
        const array = new Uint8Array(16)
        window.crypto.getRandomValues(array)
        localStorage.setItem('brian-chat-session', (session = Array.from(array).map(byte => byte.toString(16).padStart(2, '0')).join('')));
      }


      let result = await fetch('https://brian-chat.pryor.games/embed?session=' + session + '&t=' + Date.now(), {
        method: 'GET'
      })
      document.querySelector('#chat-program').innerHTML = await result.text()

      setTimeout(() => {
        let messageHistory = document.querySelector('#messages')
        messageHistory.scrollTop = messageHistory.scrollTop + 10000
        document.querySelector('#chat-program textarea').addEventListener('keyup', sendHandler)
        document.querySelector('#chat-program button').addEventListener('click', sendHandler)
        // TODO:
        //document.querySelector('#chat-program #messages').addEventListener('scroll')
      }, 100)

      // in case it takes too long to respond and it errors out in 30 seconds
      setInterval(async () => {
        let messageHistory = document.querySelector('#messages')
        let messagesWithIds = document.querySelectorAll('#messages > *[id]')
        let messagesWithoutIds = document.querySelectorAll('#messages > *:not([id])')
        if(messageHistory.scrollTop + messageHistory.clientHeight == messageHistory.scrollHeight) {
          messageHistory.scrollTop = messageHistory.scrollTop + 10000
        }
        if(!messagesWithIds[messagesWithIds.length - 1]) {
          return
        }
        let from = parseInt(messagesWithIds[messagesWithIds.length - 1].id.replace('id-', ''))
        let result = await fetch('https://brian-chat.pryor.games/embed?session=' + session + '&from=' + (from + 1) + '&t=' + Date.now(), {
          method: 'GET'
        })
        let newMessages = (/<div id="messages">([\s\S]*?)<\/div>\n<\/div>/gi).exec(await result.text())
        if(newMessages && newMessages[1].length > 1) {
          for(let i = 0; i < messagesWithoutIds.length; i++) {
            messagesWithoutIds[i].remove()
          }
          messageHistory.innerHTML += newMessages[1]
        }
        setTimeout(() => {
          if(messageHistory.scrollTop + messageHistory.clientHeight == messageHistory.scrollHeight) {
            messageHistory.scrollTop = messageHistory.scrollTop + 10000
          }
        }, 100)
        /*
        let messageHidden = document.querySelector('#messages-hidden')
        if(newMessages) {
          messageHidden.innerHTML = newMessages[1]
          setTimeout(() => {
            for(let i = 0; i < messageHidden.length; i++) {
              for(let j = 0; j < messageHistory.length; j++) {

              }
            }
          }, 100)
        }
        */
      }, 5000)

    })
  </script>
</body>

</html>


brians resume in html?


In [ ]:
<h2 style="text-align: center;">Brian Cullinan</h2>
<p style="text-align: center;"><strong>12205 Rustler Rd<br />
    Flagstaff, Arizona 86004<br />
    bjcullinan@gmail.com<br />
    +1 (480) 466-0856</strong></p>
<hr />
<p><strong>Citizenship:</strong> Yes<br />
  <strong>Special Hiring Authority:</strong> Person with Disability<br />
  <strong>Federal Experience:</strong> None (on-going?)<br />
  <strong>Clearance:</strong> Level 0 (internet?)
</p>

<!-- To obtain a temporary position in the highest office as a target -->
<p><strong>OBJECTIVE:</strong> To obtain a full time position in public service with the FBI as a residential field
  agent.</p>

<p><strong>SKILLS SUMMARY:</strong> Fast context switching, easy to focus, quick learning individual. Over 24 years
  of programming experience in <u>Computer Science</u>, <u>computer programming</u>, <u>web technologies</u>,
  <u>user-experience development</u>, <u>and data managements</u>. Over 10 years of <u>multi-cultural study</u>,
  <u>philosophy</u>, <u>cognative bias</u>, <u>psychology</u>, and <u>theology</u>. Multi-lingual with extensive
  understanding of the <u>English language</u>, and a basic understanding of <u>Polish</u> (Mandarin and Arabic
  pending). Deployed multiple business ventures complete with <u>monetization</u> on <u>cloud platforms</u>, as well
  as <u>self-hosted</u>, on <u>premise solutions</u>. I am driven, <u>highly trained</u>, <u>technologist</u> and
  <u>contradictarian</u>, with an emphasis on compassion and empathy towards others.
</p>

<!-- am i the only person in the world to write this based on personal experience instead of professional experience? -->
<ul>
  <li>In my role as a <u>Software Engineer</u> for a start-up medical company, EPIC Research and Diagnostics, I
    exercised the full extent of my <u>Computer Science</u> expertise. During my tenure I was involved in
    <u>computer programming</u> a medical kiosk, this simplified the workflow and necessary security components by
    heavily restricting access to internal systems on the Windows based workstation accompanying the medical
    scanning device. This was a homebrew <u>user-experience</u> that tailored low-level Windows API function calls,
    as well as Group Policy to prevent tampering, misuse, and common web-browsing security flaws throught the
    clinical trial.
  </li>

  <li>As a <u>technologist</u> Using MatLab native compilation, linear regression, and kirlian imaging, I spent
    months studying and rewriting the entire software platform. I was able to reduce image processing time from 10
    minutes to only 10 milliseconds per image. That is a 6,000% performance improvement over the previous version.
    The improved platform was fast enough to run on video image capture from the device. In regard to <u>data
      management</u> in no small part, the performance was improved by systematically redesigning the database model
    to use "near" 3rd-Normal form.</li>

  <li>In additional to these major business goals, I developed, along-side another colleague, an <u>on premise</u>
    internet-based solution for a personal "cloud" service that communicated locally with many connected client
    across multiple physical hospital locations. Doctors would be able to communicate test results as they work from
    different locations in town through the web based (<u>web technologies</u>) service (Microsoft MVC). This
    inspired my propensity for medicine and full-body health and continued research in the field.</li>

</ul>
<p> <br /> </p>
<ul>

  <li>With a development mindset towards body and health, I turned my focus, skill and study to the mind itself.
    A former Marine
    approached me with an idea to improve studying for college students. Over the course of 3 years, we designed and
    developed 3 individual products and <u>user expereiences</u> with modern <u>web technologies</u>. We used Drupal
    and Symphony to create an entire course based on study techniques students could implement to improve memory
    retention.</li>

  <li>Throughout working on the platform, I also benefitted from the lessons. I learned how to remove distractions
    from my environment and to vary my location while studying to expand <u>associative memory</u>. I also learned
    <u>interleaving study</u>, which is studying different topics in seqence to improve memory retention.
  </li>

  <li>I helped <u>monetize</u> the platform by configuring and implementing the proper server transactional API
    calls to Authorize.NET. This non-trivial process includes properly configuring server architecture to adhere to
    PCI-DSS (Payment card industry, data security standards). Even though our server was designed NOT to store
    customer information, we felt it was important to secure any association with student names and study data.</li>

  <li>I have adopted a mode of <u>teaching to learn</u>, which states that in order to fully understand a subject, a
    person should be able to teach it. I intend to apply this to all facets of my career. I also learned a mode of
    study popular in the medical field know as <u>spaced repitition</u>. This was the focus of our third
    application, an iOS and web-based app that implemented flash cards at specifically timed intervals. The outcome
    is that by studying flash cards that we not remembered more frequently, after a month of study the information
    can be <u>actively recalled (i.e. the testing effect)</u> from memory.</li>

</ul>
<p> <br /> </p>
<ul>

  <li>To further my studies, I spent a significate amount of time learning and recognizing over 200 cognative
    biases. In order to understand my own mental development, I explored mythology from the 5 major <u>world
      religions</u>. In order to understand the effect of religion on culture, I studies <u>cultural norms</u> such
    as whether it is appropriate to be late for a meeting, or whether I should repair an old toy versus throw it
    away and buy a new one. I extended this study into <u>cooking</u> and attempted to make dishes from major
    cultures around the world.</li>

  <li>With a basic Sumarian level understanding of moral law, I turned my focus the abuses and self-abuses of modern
    law. It became clear to that Americans are not willing to extract any measurable utility from my own mind and I
    would have to work alone. In my investigation I studied <u>anti-social behavior laws</u> as it relates to
    attempted mitigation
    through <u>shadow banning</u> on social media platforms. It was my intention to philosophically prove the same
    feature-set exposed by the darkweb, spreading <u>manipulation and lies</u> through the use of automation tools.
    These abuses finally appeared as the Cambridge Analytica scandal, and I had proven to myself and my colleagues
    how easy it was to betray people's trust, in my own small way. I asked multiple friends to "send compliments" to
    the same person, automatically using the "Like"-button on thousands of posts, automatically inviting people to
    charitable causes, and finally, scraping private message history from Facebook.</li>

  <li>Further investigation in to companies, I uncovered more <u>rule breakers</u>. Corruption from greed in public
    sector was the megalomaniac version of small-business corruption. It appeared to me that nobody felt obligated
    to
    follow the established rules. These abuses were mostly in regard to data-privacy and data-integrity laws.
    Software companies
    that actively work against their customers / client to "double dip" in the free capitalist market. <u>Selling
      personal data</u>,
    manufacturing false documentation, falsifying records, pay-walls, data "lockout", extortion, <u>lying by
      ommission</u>, gross negligence, health code violations, <u>identity theft</u>, systemic racism in the
    workplace /
    State level, dismantling of <u>homestead laws</u>, were a few "close encounters" with Justice. Unfortunately
    many
    of these are still unmitigated. The struggle is real, indeed.</li>

  <li>As a residential host to nurses, foreign travelers, and local people, I learned <u>acceptance</u>,
    <u>tolerance</u>, <u>patience</u>. I worked to overturn a living space in under 20 minutes, dining etiqette, and
    learned about many different cultures from visitors. I learned landording laws, but more importantly, I learned
    <u>stewardship</u>, as I tried to make my humble little plot profitable. I learned <u>horticulture</u> as it
    applied to growing food in a garden and transforming sand in to soil through <u>composting</u>.
  </li>

  <li>I created a lush
    garden in the desert that had no problem withstanding the hot Arizona summers. I studied municipal doctrine as
    it related to
    home ownership, and I documented minor failures of this society such as personal theft, noise ordinance, traffic
    and commercial influence as the neighborhood grew around me. </li>
  <li>I also documented the technological advancements I
    observed such as implementing street cameras at numerous intersections, implementing wireless technology for
    monitoring sewage and gray water storage, <u>traffic monitoring systems</u> installed at night across the road,
    and even the advancement of Gigabit fiber.</li>
  </li>
</ul>

<hr />

<p><strong><u>EMPLOYMENT HISTORY</u></strong>
  <br /><br />

  <strong>Sitewat.ch<br />
    Front-end Developer<br />
    $10,000 over 1.5 years</strong>
  <br /><br />
  As a Front-end developer at Sitewat.ch I implemented various interfaces for use with a web-security scanning tool
  similar to pentest-tools.com required by Authorize.NET. I implemented web-safe standards according to Owasp.org,
  and presented customers with a unified <u>user experience</u> for viewing possible vunerabilities in their
  platform. My employer applied for many <u>CVEs</u> and credited all three team-mates on <u>Google Security
    Hall-of-Fame</u>.
</p>

<ul>
  <li>Implemented a front-end interface for customers following Owasp guidelines such as safely escaping
    "<u>sinks</u>",
    anywhere in the application that a "user input" such as profile fields, or parts of the URL are inserted by into
    the page for user feedback. This would prevent a user from sending a forged (<u>XSRF</u>) link meant to trick
    the user
    into clicking on something malicious (<u>XSS</u>).</li>

  <li>Participated in a DEFCON presentation of the effort, website vulnerability testing as a service using Nexus,
    and the
    individual contributions of my employer to the field. Revealing dozens of new exploits, and existing
    vulnerabilities on major public service platforms.</li>

  <li>Used modern (at the time) design principals such as responsiveness such as resizing elements on the page for
    different devices and high contrast. Implemented Accessibility features, such as using alt-tags to describe
    content for screen readers, tab-indexes, i18t internationalization for over 50 languages. Authentication
    features such as OAuth and HTTPS for secure login.</li>

</ul>


<p><strong>EPIC Research &amp; Diagnostics<br />
    Software Engineer &amp; Network Administrator<br />
    $200,000 over 2.5 years</strong>
  <br /><br />

  As a software engineer for a <u>start-up</u> medical company, I was required to "wear multiple hats". This meant
  including myself in business planning meetings, financial discussions, <u>option vesting</u>, military contract
  proposals, <u>FDA DNVO</u>
  submissions, data integrity, <u>formal process</u> improvement. <br /><br />
  I started by creating a data entry database to migrate written records to digital form using Microsoft SQL server
  for storage and querying. Data was entered by workers from a temp-hiring agency, so I provided basic level
  technical support, any time there was confusion for a field, basic training, and user account creation through
  <u>>Active Directory</u> (Microsoft <u>LDAP</u>).<br /><br />
  I wrote a kiosk to use along side the medical device to minimize network risk and simplify workflow for hospital
  technicians.<br /><br />
  I implemented a Windows Business Server, complete with Exchange, networked file access, file-system revision
  history, <u>RAID mirroring</u>, <u>Exchange email</u>, <u>Active Directory</u>, <u>Group Policy</u> coinciding
  with a network policy that adheres to <u>CFR 21 Part 11</u>, general ITSM support, rackmounted servers for job
  processing and on-premise "cloud"-like virtual machines with <u>Hyper-V</u>, <u>Microsoft SQL Server</u>
  Finally, a colleague and I implement a "cloud"-like medical data record server in <u>Microsoft MVC</u> (a
  successor to Master pages). This service assigned a geographic location to each medical device in the field, and
  synchronized data between the client facing kiosk and the on-premise backend. The web-based system also presented
  doctors with a portal to log in and review patient records remotely. <br /><br />

  Patient records were displayed similar to the kiosk. At that time, I decided it would be a huge benefit to the
  business to rewrite the client software using <u>Window Presentation Foundation</u> controls and remove some of
  the 3rd party libraries. I was also able to reduce the processing time from 10 minutes to 10 seconds, and process
  videographic snapshots of the patient scan. I wrote an <u>automated validation</u> process that tested the same
  images with old and new versions of the client software. I used <u>Selenium</u> to validate the user experience of
  the front-end web interface.
</p>

<ul>
  <li>Participated in business development and directional meetings such as applying for FDA approval using the DNVO
    process with a precusor device called the Asira, which uses electrodes to communicate mitochondrial stresses on
    the body.</li>
  <li>Implemented an unmaximised Small Business Server with Microsoft Exchange to minimize hosting costs and secure
    intellectual property. Configured <u>Active Directory</u> and <u>Group Policy</u> to provide ubiquitous
    <u>file-system revision history</u> across all connected systems.
  </li>
  <li>Wrote a kiosk <u>explorer shell</u> to minimize security risk and simplify workflow for device technicians in
    the hospitals the company contracted with. Developed an automated and manual formal test process for validating
    the new platform.</li>
  <li>Rewrote the main client application directly connected to the device, and assisted in writing web-based
    medical record service for doctors to view patient information. Used <u>Selenium</u> for automated testing and
    validation.</li>

</ul>

<p><strong>Study Sauce<br />
    Principal Engineer<br />
    $300,000 over 3 years</strong>
  <br /><br />
  During my tenure, the owner and I designed and developer 3 unique platforms. The first platform was primarily a
  marketing and sales front-end to test the <u>target market</u> for the <u>minimally viable product</u>. Students
  would enter an individual, authenticated interface and enter their class schedule. When finished, we converted
  their schedule to a comprehensive week plan complete with when they should study each topic. This was to
  demonstrate and teach the student about time / work / study management, and encourage <u>interleaving</u>.
  <br /><br />
  The second product expanded on the studying idea by making the class schedule the first step in an online learning
  course. The course had 15 individual lessons, each lesson used <u>gamification</u> by providing a video tutorial,
  asking a few simple quiz questions, and then giving the student a badge reward. This was meant to incentivize the
  student to come back and complete future lessons. We also implemented <u>A/B marketing</u> using Google marketing
  goals. This presented a specific percent of our traffic with two different landing pages based on the concept we
  were testing.
  <br /><br />
  The third product was an <u>spaced-repitition</u> system. "[We wanted to make somethig fun]". This was an app that
  worked
  natively on iOS and had a matching web-based <u>user experience</u>. The app kept track of progress and then
  synchronized with the backend service for persistent database storage. The web-based administration offered
  teachers a way to change content that was then synchronized to every student using the app client.
  Spaced-repitition is a technique Jeopardy players and medical students alike use to memorize a lot of content
  about a subject. It works by presenting the entire set of information on the first day of use, then only
  presenting wrongly answered flash cards on the subsequent days of use.
  <br /><br />
  That is, if a card is answered incorrectly,
  it's presented the next day, if the answer is correct, the next day is skipped and it is reviewed 2 days later, if
  it is answered correctly again, it isn't presented until day 5, day 10, day 20, day 30, and so on. By spacing out
  when the
  correct information is reviewed and increasing the freqeuncy for incorrect answer, this improve memory retention
  and the ability to recall the correct answer quickly.
</p>

<ul>
  <li>Built a <u>minimal viable product (MVP)</u> using the <u>Drupal</u> platform. Created <u>A/B marketing</u>
    pages to cater to our target market. Organized student schedules into a weekly plan in PDF format. Self-hosted
    website running on a personal Linux (LAMP) server.</li>
  <li>Built a learning course using Symphony. Built 15 individual lessons using <u>gamification</u> reward system to
    keep students interested. Built 6 different tools to get students to think long-term about goals and short term
    organization in their weekly schedule. Integrated the weekly schedule with Google Calendar API.</li>
  <li>Built a <u>spaced-repitition</u> quiz app for memorization and <u>active recall</u> learning. Imported dozens
    of lesson plans coiciding with elementary learning from BASIS schools. Imported a learning set for the US
    Citizenship Test, requiring a basic understanding on American history. </li>
  <li>Simultaneously devepoled an iOS app and web-based <u>user experience</u> to make better use of Apple's iOS
    store marketing pipeline. Connected in app content to Authorize.NET to <u>monetize</u> learning content. This
    was intended to by a selling point for <u>B2B (business-to-business)</u> contracts.</li>
</ul>

<p><strong>Charles Schwab<br />
    Software Engineer<br />
    $60,000 over 4 months</strong>
  <br /><br />

  In a short sprint at this company, I began to observe oddities of corporate behavior. In between being nagged by
  my freshly assigned project manager over <u>elusive "employee training"</u>, and other managers about "what firm
  do your represent?", I investigated the company's corporate structure. Their internet "social network" was
  anything but social, built on <u>Microsoft Sharepoint</u>.
  <br /><br />

  Internally, attitudes we moot, people came to work, kept to themselves, formed alliances with people who spoke the
  same language. The building was structured more like a money-data vault with very little sunlight. The basement
  floor I was assigned to was full of <u>anti-social</u> contractors. Champagne Charlie could afford to keep me
  around but I was let go for my "attitude".
  <br /><br />

  While trying to rationalize why such a significant company would intentionally <u>"Manage me out"</u> (yes, this
  is a real strategy), I had my first experience with Toast-masters. This seems like a great way for employees to
  get to know one another, and learn about <u>public speaking</u>. Charle's Schwab apparently offered "learning
  tracks" to their employees and would sponsor testing and certifications. In particular a track to becoming an ITSM
  "security manager" with zero coding experience.
  <br /><br />

  One of the more exciting events was a <u>"Kaizen" workshop</u> formed for employees. This was a good way to
  introduce <u>continuous integration</u> style practices, in a formalized way, to people who haven't studied
  <u>Agile</u> and require lots of structure. Tried not to take offense when another employee passive-agressively
  expressed that I "needed to get up to speed". Out of this meeting, my team was inspired to brainstorm ways their
  project applies to business needs. This was great because I had a lot of experience <u>brainstorming</u> outside
  of the basement, but all ideas during the session were still confined to a very small box in the form of a
  window-less conference room.
</p>

<ul>
  <li>Learned the corporate structure through a Microsoft Sharepoint based social networking application. Helped
    managers identify "key stake-holders" for supporting our project's effort. Integrated with "big personalities"
    by being friendly and trying not to pry into previous experiences.</li>
  <li>Realized one of the major reasons their simple project, "remove secondary access to databases through an
    internet request tool" was so difficult to accomplish because they insisted on using Rational Unified Process.
    Charles Schwab heavily relies on IBM and RUP is an IBM product.</li>
  <li>Tried to navigate "big personalities" from a tremendous project manager as I asked to help with timeline
    goals. Tried to retrain my managers to approach RUP holistically, that is, all documentation should have been
    developed simultaneously, rather than sequentially. The methodology in RUP doesn't make this clear, but drawing
    from past miseries using the process at the medical company, I saw this as a major failure point.</li>
  <li>Failed to communicate the importance of brainstorming topics and diversifying your environment as a means to
    creatively solving a problem. Introduced managers to a document control system (DCS). Configured a Microsoft
    Sharepoint site to link to "artifacts" within the DCS.</li>
  <li>By the end of the experience I had realized I was being intentionally ignored as a part of an external growth
    process. Thank you to Charles Schwab for being large enough to bank-roll my failures. I was also emotionally
    detached when I realized an 80 year old mainframer came to work with an eye infection instead of taking care of
    herself and retiring (since she worked for a retirement company), I viewed this as flagrantly hypocritical and
    sad.</li>
</ul>


<p><strong>Swiftpage<br />
    Senior Software Engineer<br />
    $100,000 over 14 months</strong>
  <br /><br />

  The same day of my release from my previous company, I was invited to interview at a <u>small-business</u>
  marketing company. This was particularly interesting because my father's manufacturing company is a client. I saw
  this as both a way to better understand marketing, and even improve upon a product his company relied on. With his
  advice to "work for a small-business", I was excited to contribute.
  <br /><br />

  During my tenure, realizing my propensity for <u>quelling disorderly behavior</u>, I was as minimally disorderly
  as possible. I tested networking features by using reddit.com/NSFW from the bathroom. They did indeed implement a
  <u>censhorship</u> system on their internet connection, the results were abstract to me, something I'd like to see
  changed for <u>remote-workers</u> and <u>personal-accountability</u>.
  <br /><br />

  Human Resources implemented a policy for "unlimited paid-time-off". This is a psychological trick to get people to
  take less time off, feeling like it's always there when they need it. I worked an entire year, about 3,500 hours
  learning NodeJS without taking a single day off, even during the Christmas holiday when most of the office was
  empty. Ironically, when I first entered the company, a woman was begging for time off over conference calls. This
  is a failing policy and people should be compensated fairly for their time.
  <br /><br />

  The development process consisted of employees trying to avoid an antiquated megolith written with overly burdened
  state-management design principals ("<u>feature completness</u>", see Microsoft Code-Complete). When a change was
  integrated the system would need to be update after work hours on Wednesday nights. If the change caused a failure
  in the test-automation suite that was both precious and lacking "<u>integration tests</u>" the entire change would
  be rolled back and attempted again the next night. It was the absolute worse possible method of avoiding
  <u>continuous integration</u> (in software, solutions were widely available from major distributors).

  The company offered a "learning track" and modes for self-improvement and even made their Job description sound
  like they would pay for, or contribute to my education. At one point LinkedIn Learning was advertised to employees
  in the office, and they offered sponsorship. I spent the entire month programming on the work project, and using
  my secondary monitor to absorb programing and management courses on LinkedIn. When the month trial period expired,
  LinkedIn Learning billed my personal account instead of the business account I had joined through.
  <br /><br />

  Eventually, the work became too much, when I witnessed other managers in the office yelling at my boss to change
  the pricing model to perpetuate the multi-level marketing sales. I offered to take my boss to lunch after he told
  me that "the company has high expectations of me" without formalizing or writing those expectations down. Employee
  reviews were conducted through ADP, at the beginning of the year he asked me for "what goals I would like to
  accomplish". These goals were never readdressed, and at the end of the year, the goals my boss wrote into ADP had
  nothing to do with the goals I sent him at the beginning of the year. My boss criticized my in front of other
  employees in addition to the criticism in our monthly one-on-one meetings. I had enough, I was being "Managed
  out".
</p>

<ul>
  <li>Built a marketing front end that integrates with Zuora (the billing platform like Authorize.NET). Used Angular
    JS to create a new login form, account settings, <u>buy funnel</u>, billing statement (PDF generated by Zuora),
    and itemized purchase page.</li>
  <li>Integrated Zuora with Eloqua, an enterprise email marketing solution from Oracle (similar to SendGrid). Failed
    to explain to my boss Amazon AWS doesn't support C# unless it is built on .NET Core and would require a rewrite
    of the entire platform. Lost a month of effort and switched to NodeJS which was much more widely support on AWS.
    Used API calles of both Zuora and Eloqua to create a <u>micro-service</u>, or <u>serverless function</u> (not
    sure of the correct name, lots of terminology was being thrown around at the time)</li>
  <li>Formed alliances with the marketing team that was competing for the same ownership over the product sales
    feature of the customer facing website. Validated and automated a manual process costing the company no less
    that $10,000 per year. It felt awkward to change teams within the same company, so I didn't apply for
    alternative roles they advertised even thought they matched my skill-set.</li>
  <li>Realized I was being <u>Managed out</u> when my boss told me the effort working with multiple marketing
    managers to automate and validated a manual billing process was "a waste". I saw the effort as a simple
    equation, if a company spends $10,000 per year on a manual process, and pays me to automate the process, it is
    somewhere between 1 - 2 years of operating automatically before they save and repurpose the money that would
    have been spent on manual labor. The employees involved we all ecstatic to never have to do this again, but my
    boss couldn't be concinved.</li>
  <li>For the first time in my life, I felt I needed to take control of my life, instead of just taking things as
    they come. I quit. It was obvious people didn't want to communicate with me decently.</li>
</ul>


<p><strong>Sabbatical<br />
    Indentured Servitude<br />
    $60,000 over 2 years</strong>
  <br /><br />

  Realizing my mind wasn't fit for corporate or the small-business world, I turned my study to something more
  philosphical. I tried to get to know people through hosting. I applied for a job in San Franscisco with digit.co.
  I realized quickly the financial sector would not be kind to me.
  <br /><br />

  I left the US for 3 months to live in Poland. I cashed out all my credit cards. I learned what it meant to live
  with a packed suitcase. I studied philosophy, The Bible, Hinduism, Buddhism, Judaism, and spiritual religions. I
  navigated a foriegn country with my wife, I was amazed by how things had progressed having not been to Europe for
  10 years. Internet was widely available, dozens of American services like Uber and Airbnb were widely
  available. I was amazed that <u>censorship</u> of American TV programs was still common practice. Despite being an
  America and paying for HBO, I wasn't allowed to watch the same HBO programs in Europe.
  <br /><br />

  I became a father in Poland after American doctors told my wife that she would have a natural birth if they
  decided that was the safest method of delivery. I saw a pattern of <u>abuse of authority</u> when doctors now
  promote cesarean surgeries. They can collect more money from insurance companies, not because the recovery or
  delivery is safer. Doctors told my wife that "In America the doctor decides what is best for you, and if you don't
  like it you can go back to Poland." after she requested an episiotomy. Doctors in America don't practice medicine
  once they leave college and get to a specific level of authority.
  <br /><br />

  I Poland we enjoyed many beautiful gardens. My wife enjoyed speaking her native language and I enjoyed learning
  it. She taught me about history, and I realized I could stand on a side-walk that was older than all of United
  States history. I was catered to at hospitals. I walked in one day with a kidney pain and was able to talk to a
  doctor immediately, they prescribed a rosemary oil based medicine and I didn't owe thousands or even hundreds of
  dollars for the visit.
  <br /><br />

  The doctors treated my wife with respect. They didn't question where she came from, they did ask why she came to
  Poland and we explained how doctors have no problem betraying people's trust in American medicine. The hospital
  refunded the money we spent on consultations we had before paying for a birthing package. This was absolutely
  astounding compared to Phoenix, Arizona doctors that wanted nearly $20,000 up-front with no refunds should
  something happen, that price does not include the hospital bill. We were able to pay for child delivery out of
  pocket.
  <br /><br />

  Airline companies act differently in Europe, almost like they want people to travel easily. The only difficulty
  was in Sweden when they required we come to the airport 4 hours ahead of time with a printed copy of our baording
  pass, the charged a $40 fine for printing out my ticket and being late after my wife argued with them not to
  cancel our flight. This was a flagrant abuse of power and the women behind us was not so lucky, they cancelled her
  flight. We flew Nordic Airlines before it was renamed to Norse Atlantic because of American politics not being
  able to compete with a government that subsidized flights thanks to being a large oil-refining country.
  <br /><br />

  We encountered systemic "dragging of feet" visiting government offices and nagivating paperwork. We completed a
  Consulary Report of Birth Abroad, and gained United States and <u>Polish citizenship</u> (<u>dual-citizenship</u>)
  for our first born son within
  the month following his birth. I realized in the hospital that I'm living in a time period where circumcision
  could be considered a "form of abuse", this weighed on me heavily, but ultimately after researching the genitals
  couldn't be cleaned until 3 years of age, we decided to circumcise. This was an "uncommon request" hospital staff
  explained, but they were able to source a female Jewish surgeon from New York. I'm amazed at how perfectly well
  our trip went despite politics making it appear as though The Old World is behind America technologically.
  American doctors, unfamiliar with Noam Chomsky's opinions on American Exceptionalism, explained to us that America
  is the best place in the world for medicine. I've laid this false information to rest, it is banter and nothing
  more, people are mostly the same everywhere, some ambitious and educated, some lazy and avoiding work, and some
  happy to do food deliveries with American software platforms.
  <br /><br />

</p>


<ul>
  <li>Discovered Poland an amazing American and Schengen ally. Practiced the <u>Polish language</u> I learned
    through Duolingo. Consulted with doctors on child birth. Confronted government paperwork with agility and
    punctuality. Avoided having my marraige certificate stolen from me with a side note of casual rascism directed
    at my wife, stemming from <u>Jewish inheritance</u>. Lived happily, and nearly applied for a job setting up
    rseidential
    A/V home automation equipment in Wilonow, a suburb of Warschawa.</li>
  <li>Learned about child birth, episiotomy, and catering to a new-born and a wounded wife. Thanks to the surgical
    precision of the episiotomy my wife was able to heal and nearly stop bleeding after only 3 days of discomfort.
    She had her "disolvable" stitches removed after 3 days because they poked her and the bleeding already stopped.
    I cooked for her every day in a small apartment loft booked through Airbnb.
  </li>
  <li>Navigated government offices and paperwork to complete applications for a United States and Polish
    citizenship. Gained <u>dual-citizenship</u> for our son, and felt a great reward from having the Consular Report
    of
    Birth Abroad. Felt like a diplomat visting multiple government offices. In Poland, asking a government worker to
    do work felt more like asking for a favor, where in America paying a fee feels more like an expectation that
    work will get done.</li>
  <li>Visited gardens and absorbed Polish history through visiting old shops, and castles downtown. Relearned how to
    enjoy people. Learned about cooking and philosophy. Learned about religion and politics as "Seperation of Church
    and State" is an American ideal. In Poland, government offices are heavily influence by Cathoicism, and that
    affects policies like abortion being illegal (2022 in Arizona this seems relevant). In American politics,
    seperation of church and state is an "ideal", ideal means ideally, politics are fair for all beliefs, but that
    isn't always how law turns out in practice.</li>
  <li>Realizing this distinction, I turned my foundational knowledge of religion towards to law. I tried to search
    for philosophical dichotomies where the law doesn't match "ideal" situations, where the law has a short-coming
    of morality. Personally, I wanted to understand my own experience with the law as it related to being lied to
    over a Solar leasing program from Tesla (then SolarCity owned by Elon Musk's brother in law).</li>
</ul>


<p><strong>idTech 3 Game Engine<br />
    Game Engine Developer<br />
    $60,000 over 2 years</strong>
  <br /><br />

  When I first approached this game-engine, I intended to improve the loading experience of QuakeJS. QuakeJS is a
  Web Assembly / emscripten port of Quake 3 to the web-browser. The loading process took about 10 minutes over
  cellular network, and desktop was not much better. It first loads the Trial Demo content, about 300 MBs of data.
  Then it loads additional game data to run custom levels ("maps" as they are called in game).
  <br /><br />

  My approach was to minimize the number of line changes stylistically. That is, rather than moving and renaming
  functions, I would use a pre-compiler macro to create the necessary logical breaks in the function, then call the
  new broken up function in the same sequence that QuakeJS did. Since I already had a working implementation, it was
  easy for me to see if my change caused a breaking error, or if it was indeed calling the function just like the
  original port, only with less code changes.
  <br /><br />

  Once completed with the code changes, I iterated on development trying to find the most burdensome pieces of code.
  i.e. The parts that added the most complexity through new variables, lots of line changes, bug reports,etc. One of
  the biggest hurdles was this idea of a file manifest.json. This was hard to create, and people ran into errors
  trying to generate additional content through the repack.js script. The author's intention was to repackage game
  assets to be smaller and compatible with web. I reapproached this issue 3 times during my development.
  <br /><br />

  Once I had a solid foundation and a working Web Assembly build, I attempted to distill out of emscripten only the
  system-level APIs that it required to run this particular engine. I expected to double the performance, and that's
  exactly what came from the effort. By removing the complexity implemented by emscripten's burdensome file-system
  APIs, extra work being done by the GL emulator, and memory leaking from Audio spatialization with SDL2, I was able
  to double the performance. Maps that used to only run well at 30 frames per second (FPS) could now run at 60FPS
  and in-general I could run the engine at over 200 FPS inside the browser window.
  <br /><br />

  I then turned my focus to game developement, I wanted to implement a Portal system similar to prioprietary
  closed-source forks. I was able to adapt some of the code left-over from the original game that was left unused in
  the source code. I copied some of the models from another Portal implementation and implemented the missing
  functionality to display droppable portal or wall mounted portals like the game Portal from Valve. This was an
  interesting project because it required a fairly in depth understanding of renderer code and limitations enforced
  from legacy systems.
  <br /><br />

  I realized this work was much too fun for one person and I reached out to other community leaders like to owner of
  lvlworld.com with an idea to present all the lvlworld maps in 3D as a fun exploration tool for visitors. He loved
  the idea and eventually rewrote my entire effort to his own accord. I felt terrible about not being able to
  support the website anymore, but ultimately it felt like a win having another proprietary, closed source
  implementation as a good learning experience for the owner, Tig.
  <br /><br />

</p>


<ul>
  <li>Taught myself the C, C++, Web Assembly, and emscripten eco-system based on examplary work from inolen and
    QuakeJS. Learned to communicate with other community leaders. Learned there is a heavy bias towards not
    violating GPL licenses with other GPL licenses. This is a huge roadblock for software, but I understand no one
    would agree with me. They just don't see discovery and learning as being more important than respecting
    propriety. The propietors of many engine forks don't feel the need to share their work with anyone.</li>
  <li>Learned to use Discord. Assisted in other people's discovery path by answering technical questions about the
    engine. Provided line-by-line demonstrations of how I might have solved a similar problem they are facing. Tried
    to built community membership by encouraging developers to explore a variety of communities as they search for
    answers to their design questions. For example, if one community already tackled display unicode, it's possible
    the licensing would allow them to reuse code for their own open-source implementation.</li>
  <li>Contributed to open-source projects and was a member of GitHub Arctic Vault. Eventually stopped working on the
    project entirely due to legal threats. I asked community members about licensing and they were disinterested at
    best. The understanding of legality was more important than the developmental outcome of my effort.</li>
  <li>Learned a lot about vector graphics, linear algebra, graphics hardware, 3D algorithms. Learned about my own
    mind and my ability to absorb knowledge and experience from working with other people. Learned to enjoy software
    development again, it felt like I was doing it for myself instead of being told to do it for someone else like
    in the corporate or small-business environment. Learned the process of learning is sometimes more important than
    the outcome</li>
  <li>Learned to use Twitch and Youtube live streaming. Learned to use Web Assembly in a vast array of projects,
    porting software to the web browser is very fascinating to me, even though it feels like propriatary
    close-source companies are trying to destroy the free and open web. Learned to filter out which parts of my
    experience are relevant, extreme, or unnecessary.</li>
</ul>

<hr />

<p><strong><u>ACCOMPLISHMENTS</u></strong>
  <br /><br />
</p>

<!-- this ought to be fun -->
<ul>
  <li>Learned about whole body health, and how mitochondrial stressors are represented by low-level light detection
    eminated from the body (Kirlian imaging). A previously thought psuedo-science has scientific backing when
    Eukyotes "powerhouse" the mitochondria is treated like a radio signal.</li>
  <li>Learned to expand my learning through better study methods. Iterating through study techniques every day to
    exercise my eididic memory. Continued my studies using the interleaving method so my mind never gets fixated on
    a single box for too long. Increasing blood-flow and brain activity by learning construction, physical exercise,
    and new techniques in software development.</li>
  <li>Improved my spiritual health by trying to reconcile religious beliefs with scientific possibilities through
    extensive study of multiple world religion. Began to understand the patterns how many beliefs are built on top
    of other beliefs. Development a constradictarian mindset by question the belief systmes of colleagues that only
    focus on their own "1 true religion". Quelled my own personal philosphies in favor of philosphy learned versus
    philosphy implemented in my behavior.</li>
  <li>Learning game engine specifics like physics interactions with many entities, 3D geometric spaces, linear
    algebra, familiarity with floating point hardware (GPUs), graphics processing pipelines, Web Assembly, and the
    C, C++, Makefile eco-system.</li>
  <li>Read and familiarized myself with dozens of statutes including Arizona Landlord/Lesser laws, CPPA (California
    Privacy Act), GDPR (European General Data Protection Regulation), CFR 21 Part 11, PCI-DSS (Payment Card Industry
    - Data Security Standards), Title 47 (Federal Communication Regulation pending), Arizona Traffic Laws (in it's
    entirety), Arizona Property Statutes (Title 33), HIPAA, tax regulations, dozens of apriori relevant to main
    stream outcomes, dozens of encyclopedia entries related to law and regulation.</li>
  <!-- some people that experience this level of fear kill themselves, remembering https://en.wikipedia.org/wiki/Etika and https://en.m.wikipedia.org/wiki/Aaron_Swartz their end came too soon before the light could reach them, also remembering Goysha, my wife's friend was as brilliant as they come, sometimes the saddness in the world is too much for people. Personally, I can't earn an honorary doctorate of psychology, or a nobel prize if I'm dead, prehaps this gives me transhumanist ambition? I said to Stephen, "this world is too small for a man of my ambition", a quote from Big Fish. The devil is in the detail, i'm very detail oriented, i don't see this as ironic, it's very intentional, this was planned for a long time, God is a programmer, indentured servituted is my punishment for treating the Justice Department like a confessional. -->
  <li>Overcoming childhood trauma and prayer as a methodology for improving memory retention (in progress). Taking
    notes on oscillating thought every second. It appears that thinking about possibility or planning the future is
    the opposite of living in the moment. Possible expansion of intuition by oscillating between possibility and
    then focusing on the moment, blurring vision of current scenery, then focusing on nothing, even the word
    "nothing" in my mind to detach. Hard to oscillate at 60 BPM, might be the key to controlling theta waves.
    Putting slight pressure with my hand on the back of my head helps to clear thoughts, might help with motion
    sickness. Indictation not to pop my knuckles or crack my joints, conflict with chiropracty (in progress)?</li>
</ul>


<p><strong><u>COMPUTER SOFTWARE</u></strong>
  <br /><br />
</p>

<ul>
  <li>NodeJS - 10,000+ hours - Expert</li>
  <li>Linux, Windows, MacOS - 10,000+ hours - Expert</li>
  <li>C, C++, Makefile - 10,000+ hours - Expert</li>
  <li>PHP - 10,000+ hours - Expert</li>
  <li>English - 10,000+ hours - Expert</li>
  <li>Web-based UX design - 10,000+ hours - Expert</li>
  <li>Web-based Validation - 10,000+ hours - Expert</li>
  <li>Time Management - 10,000+ hours - Expert (just ask to see my calendar)</li>
  <li>Studying Demons - 10,000+ hours - Expert</li>
  <li></li>
</ul>


<p><strong><u>ADDITIONAL TRAINING</u></strong>
  <br /><br />
</p>

<ul>
  <li>120 hours of LinkedIn Learning Management Courses</li>
  <li>10,000+ hours studying Philosophy, Psychology, and Theology (I'm like an encyclopedia of barely useful
    information)</li>
  <!-- yeah, this is where I'm expected to go to hell quietly, Tesla didn't have an inheritance so maybe I can reverse this attitude spread widely across the field of Computer Science and EE -->
  <li>The full ET experience thanks to Nikola Tesla.</li>
</ul>



<p><strong><u>EDUCATION</u></strong>
  <br /><br />

  BA - Bachelor's of Science in Computer Science, Northern Arizona University, Flagstaff, Arizona, 2010 (GPA 2.5)
</p>


<p><strong><u>HONORS AND AWARDS</u></strong>
  <br /><br />

  None
</p>



<p><strong><u>VOLUNTEER WORK &amp; COMMUNITY INVOLVEMENT</u></strong>
  <br /><br />

  ACM - Association for Computing Machinery<br />
  IEEE - Institute of Electronics Engineers
</p>